In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import sys
import scipy
sys.path.append('/Users/ruby/EoR/pyuvdata')
from pyuvdata import UVData

In [68]:
# Load data from pyuvsim simulation:
path = '/Users/ruby/EoR/compact_redundant_array_sim_May2020'
data_sim = UVData()
data_sim.read_uvh5('{}/square_grid_sim__results.uvh5'.format(path))

In [69]:
# Inflate data
print(np.shape(data_sim.baseline_array))
data_sim.inflate_by_redundancy()
print(np.shape(data_sim.baseline_array))

(61,)
(666,)


In [70]:
# Remove autos
data_sim.select(ant_str='cross')

In [112]:
# Convert baselines to have u>0
data_sim.conjugate_bls(convention='u>0', use_enu=False, uvw_tol=0.01)

In [113]:
baseline_groups, vec_bin_centers, lengths, conjugates = data_sim.get_redundancies(
    tol=0.1, use_antpos=False, include_conjugates=True, include_autos=True, conjugate_bls=False
)

In [115]:
print(np.shape(baseline_groups))
print(data_sim.Nbls)
print(np.shape(conjugates))

(60,)
630
(0,)


In [116]:
np.shape(baseline_groups)[0]

60

In [117]:
print(np.shape(vec_bin_centers))
print(vec_bin_centers[1])

(60, 3)
[ 2.80000000e+01 -1.20564891e-10 -2.39678499e-10]


In [118]:
# This returns all the baseline coordinates for a square array with antenna separations of 1
# For an NxN rectangular array, the number of redundant baseline groups is 2N^2-2N+1
# Excluding the autocorrelations, the number of redundant baseline groups is 2N^2-2N
array_side_length_ants = 6
N_baseline_groups = 2*array_side_length_ants**2-2*array_side_length_ants
baseline_group_coords = np.zeros((N_baseline_groups, 2))
bl_group_ind = 0
for uval in range(array_side_length_ants):
    if uval == 0:
        vrange = range(1, array_side_length_ants)
    else:
        vrange = range(-(array_side_length_ants-1), array_side_length_ants)
    for vval in vrange:
        baseline_group_coords[bl_group_ind, 0] = uval
        baseline_group_coords[bl_group_ind, 1] = vval
        bl_group_ind += 1

In [109]:
baseline_cov_array = np.diag(np.full(np.shape(baseline_groups)[0], 1.))
min_bl_length = 14.
tolerance = .01
for bl_1 in range(np.shape(baseline_groups)[0]):
    for bl_2 in [ind for ind in range(np.shape(baseline_groups)[0]) if ind != bl_1]:
        bl_separation_sq = (
            (vec_bin_centers[bl_1, 0]-vec_bin_centers[bl_2, 0])**2
            + (vec_bin_centers[bl_1, 1]-vec_bin_centers[bl_2, 1])**2
        )
        if (min_bl_length-tolerance)**2 <= bl_separation_sq <= (min_bl_length+tolerance)**2:
            baseline_cov_array[bl_1, bl_2] = 0.1617
        elif 2*(min_bl_length-tolerance)**2 <= bl_separation_sq <= 2*(min_bl_length+tolerance)**2:
            baseline_cov_array[bl_1, bl_2] = 0.0176

In [119]:
# Check that the covariance matrix is full-rank
print(N_baseline_groups)
print(np.linalg.matrix_rank(baseline_cov_array))

60
60


In [120]:
print(baseline_cov_array[5,:])

[0.1617 0.0176 0.     0.     0.     1.     0.1617 0.     0.     0.
 0.1617 0.0176 0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.1617 0.     0.     0.     0.     0.     0.0176 0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]


In [108]:
print(vec_bin_centers[5])
print(vec_bin_centers[0])
print(vec_bin_centers[6])
print(vec_bin_centers[10])
print(vec_bin_centers[30])
print(vec_bin_centers[32])
print(vec_bin_centers[37])

[ 1.40000000e+01  1.40000000e+01 -2.02179606e-10]
[ 1.40000000e+01 -6.02824457e-11 -1.19840138e-10]
[ 2.80000000e+01  1.40000000e+01 -3.22019744e-10]
[ 1.4000000e+01  2.8000000e+01 -7.5267792e-11]
[-6.30251407e-11  1.40000000e+01 -8.23412449e-11]
[ 2.80000000e+01 -1.40000000e+01 -1.57337254e-10]
[ 1.40000000e+01 -2.80000000e+01 -1.64410707e-10]


In [104]:
bl_separation_sq = (
    (vec_bin_centers[5, 0]-vec_bin_centers[12, 0])**2 
    + (np.abs(vec_bin_centers[5, 1])-np.abs(vec_bin_centers[12, 1]))**2
)
print(bl_separation_sq)
print((min_bl_length-tolerance)**2)
print((min_bl_length+tolerance)**2)
print((min_bl_length-tolerance)**2 <= bl_separation_sq <= (min_bl_length+tolerance)**2)

980.0000000044953
195.7201
196.2801
False


In [ ]:
def cost_function(x):
    # x = [[real antenna gains], [imag antenna gains], [real visibilities], [imag visibilities]]